# Practical Introduction to Machine Learning
by <b>Mauricio Araya</b>

<b>Credits:</b> Francisco Foster, Matthew Graham, Pavlos Protopapas

## 1.- SDSS Data 
  
<img src="https://www.sdss.org/wp-content/uploads/2014/11/SDSS_telescope_new.jpg" alt="SLOAN" width="300">

Lets download data from the Sloan Digital Sky Survey, the all-time favorite dataset for Machine Learning in Astronomy. We could have used data from UCI or Kaggle, but I think SDSS data is very appropaite for this school ;).

## 1.1.- Download Star Photometry Data using AstroML
We will start cheating a little bit by using the AstroML package

`conda install -c astropy astroml`

In [ ]:
from astroML.datasets import fetch_rrlyrae_combined
sdss_star_feat, sdss_star_type = fetch_rrlyrae_combined()

In [ ]:
sdss_star_type

and use the Pandas package...

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)
%matplotlib inline
pd.set_option('display.max_rows',10)

as our data manager

In [ ]:
star_feat=pd.DataFrame(sdss_star_feat)
star_feat.columns=['u-g', 'g-r', 'r-i', 'i-z']
star_feat.plot.scatter('u-g', 'g-r')
star_feat

This data also have labels.

In [ ]:

import matplotlib.pyplot as plt
star_label=pd.DataFrame(sdss_star_type)
star_label.columns=['Type']

fig, ax = plt.subplots()
star_feat[star_label['Type']==0].plot.scatter('u-g','g-r',c='red',ax=ax)
star_feat[star_label['Type']==1].plot.scatter('u-g','g-r',c='blue',ax=ax)

In [ ]:
star_feat['Type']=star_label['Type']
star_feat

## 1.2.- Download Galaxy Photometry Data (with Redshifts)
This dataset are galaxies with known (spectroscopically confirmed) redshifts and colour magnitudes. We're interested in determining the redshift of a galaxy from its colors (photometric redshift). The data can be downloaded from: http://www.astro.caltech.edu/~mjg/sdss_gal.csv.gz, and we will use `urllib`to do this from the notebook

In [ ]:
import urllib
urllib.request.urlretrieve("http://www.astro.caltech.edu/~mjg/sdss_gal.csv.gz", "sdss_gal.csv.gz")
!gunzip sdss_gal.csv.gz

In [ ]:
galaxy_feat = pd.read_csv('sdss_gal.csv', low_memory=False)
galaxy_feat

In [ ]:
gal_sample = galaxy_feat.sample(n=1000)
gal_sample.plot.scatter('g-r','redshift',color='gray',alpha=0.1)

## 2.- Regression
Regression is about predicting values of continous variables.

$$y' = f(x' \mid \mathbf{X},\mathbf{y})$$

where $y \in \mathbb{R}^n$, $x \in \mathbb{R}^m$ and $\mathbf{y}$ and $\mathbf{X}$ are the target and non-target features for all the samples respectively.

We will use the SDSS Galaxy data, well... a portion of it.

In [ ]:
# Simple... not correct
train_data = gal_sample[:750]
test_data = gal_sample[750:]
y_train = train_data['redshift']
X_train = train_data['g-r']
# Formatting hack...
X_train=X_train.values.reshape(len(X_train), 1);


## 2.1.- Parametric Regression

We can condense the information found in $\mathbf{X}$ and $\mathbf{y}$ by imposing a *parametric model*, meaning to optimize certain parameters for the given data. Now our model is
$$y' = f(x' ; \theta^*)$$
where 
$$\theta^* = \underset{\theta}{\operatorname{argmax}} \left\{ Pr(Y = f(X;\theta) \mid \mathbf{X}, \mathbf{y}) \right\}$$
which under a <b>linear model</b> and a Gaussian noise $\epsilon$ assumption ($Y = f(X) + \epsilon $) it becomes
$$ \theta^* = \underset{\theta}{\operatorname{argmin}} \left\{ \sum_i (y_i - f(X_i;\theta))^2 \right\}$$.

Consider now a straight line as our model,
$$ f(x;\theta) = a x + b$$
where our parameters are $\theta = \{a,b\}$.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import numpy as np

regression = LinearRegression(fit_intercept=True)
regression.fit(X_train, y_train)

regression_line = lambda x: regression.intercept_ + regression.coef_ * x
print('The equation of the regression line is: {} + {} * x'.format(regression.intercept_, regression.coef_[0]))

In [ ]:
fig, ax = plt.subplots()
x_vals = np.linspace(0, 3, 100)

train_data.plot.scatter('g-r','redshift',color='gray',alpha=0.1,label='data',ax=ax)
ax.plot(x_vals, regression_line(x_vals), color='red', linewidth=1.0, label='regression line')
plt.legend()

Not very good... lets try another <b>linear</b> model!
$$ y = ax^3 + bx^2 + cx + d $$

In [ ]:
gen_poly_terms = PolynomialFeatures(degree=2)
X_train_with_poly = gen_poly_terms.fit_transform(X_train)
poly_regression = LinearRegression(fit_intercept=True)
poly_regression.fit(X_train_with_poly, y_train)
display(poly_regression.coef_)
poly_regression.intercept_

In [ ]:
fig, ax = plt.subplots()
coef = poly_regression.coef_
inter = poly_regression.intercept_
poly = lambda x: inter + coef[1] * x + coef[2] * x*x 
train_data.plot.scatter('g-r','redshift',color='gray',alpha=0.1,label='data',ax=ax)
ax.plot(x_vals, poly(x_vals), color='red', linewidth=1.0, label='regression line')
plt.legend()

In [ ]:
gen_poly_terms = PolynomialFeatures(degree=3)
X_train_with_poly = gen_poly_terms.fit_transform(X_train)
poly_regression = LinearRegression(fit_intercept=True)
poly_regression.fit(X_train_with_poly, y_train)
display(poly_regression.coef_)
poly_regression.intercept_

In [ ]:
fig, ax = plt.subplots()
coef = poly_regression.coef_
inter = poly_regression.intercept_
poly = lambda x: inter + coef[1] * x + coef[2] * x*x + coef[3]*x*x*x
train_data.plot.scatter('g-r','redshift',color='gray',alpha=0.1,label='data',ax=ax)
ax.plot(x_vals, poly(x_vals), color='red', linewidth=1.0, label='regression line')
plt.legend()

In [ ]:
fig, ax = plt.subplots()
test_data.plot.scatter('g-r','redshift',color='gray',alpha=0.1,label='data',ax=ax)
ax.plot(x_vals, poly(x_vals), color='red', linewidth=1.0, label='regression line')
plt.legend()

We have basically *learned* the parameters!

This is not the best we can do of course!, we can:
* Change the function/model
* Use more dimensions
* Go non-linear...
* Use more/better data
* Use regularized models
* etc...

## 2.2 Non-parametric Regression

Consolidating data into model parameters have some advantages and drawbacks. An alternative is to use non-parametric models. Now, we want to predict 
$$ y' = f(x'; \mathbf{X}, \mathbf{y}, \theta_0) $$
For example, consider a model based on assigning the same Gaussian function (Kernel) to each sample:
$$ K_\sigma(x)=\frac{1}{\sqrt{2\pi}\sigma}exp\left(\frac{-x^2}{2\sigma^2}\right)$$
$$ y'=\frac{\sum_{i=1}^n K_\sigma(x'-x_i)y_i}{\sum_{i=1}^nK_\sigma(x'-x_i)}$$
Please note that $\theta_0 = \sigma$: 

<b>Non-parametric $\neq$ no parameters!</b>

In [ ]:
import numpy as np

def GKR(x_predict,x_data,y_data,s):
    dmat = np.tile(x_data,len(x_predict))
    dmat = dmat - np.tile(x_predict,(len(x_data),1))
    K = np.exp(-(dmat*dmat)/(2*s*s))/(np.sqrt(2*np.pi)*s)
    return(K.T.dot(y_data) / K.sum(axis=0))

In [ ]:
def plot_gkr(sigma=0.1):
    y_gkr=GKR(x_vals,X_train,y_train,sigma)
    fig, ax = plt.subplots()
    train_data.plot.scatter('g-r','redshift',color='gray',alpha=0.1,label='data',s=sigma*500,ax=ax)
    ax.plot(x_vals, y_gkr, color='red', linewidth=1.0, label='regression line')
    plt.legend()

In [ ]:
from ipywidgets import interact
interact(plot_gkr,sigma=(0.01,1.0,0.01))

There are much smarter ways to do this... for example Gaussian Processes!

## 3.- Labelling

Consider now the SDSS star photometry data. 

<b>Warning:</b> we will do this *naively* (i.e., wrongly). During the rest of the week we will improve this...


In [ ]:
N=2000
star_sample=star_feat[-1:-N-1:-1]
star_sample = star_sample.sample(n=N)
star_train = star_sample[:int(N*0.75)]
star_test = star_sample[int(N*0.75):]
star_train

In [ ]:
fig, ax = plt.subplots()
star_train[star_train['Type']==0].plot.scatter('u-g','g-r',c='red',ax=ax)
star_train[star_train['Type']==1].plot.scatter('u-g','g-r',c='blue',ax=ax)

In [ ]:
fig, ax = plt.subplots()
star_test[star_test['Type']==0].plot.scatter('u-g','g-r',c='red',ax=ax)
star_test[star_test['Type']==1].plot.scatter('u-g','g-r',c='blue',ax=ax)

In [ ]:
display(star_train['Type'].sum()/len(star_train))
display(star_test['Type'].sum()/len(star_test))
display(star_feat['Type'].sum()/len(star_feat))

## 3.1.- Classification (Supervised)

Classification is labelling based on previously annotated samples.

### Discriminative Classification Models
Think on a boundary dividing data. In 2 dimensions is a line/curve, in 3 dimensions a surface, in 4 dimensions a volume, and so on. The boundary divides data into classes. This is what is called a <b>discriminative model</b>. 

#### Support Vector Machines
*Vocabulary:* This is a  <font color='blue'>discriminative</font> <font color='green'>(non-parametric)</font> <font color='magenta'>linear</font> model for a <font color='red'>supervised</font> <font color='orange'>batch</font>  learning problem

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(star_train[['u-g','g-r']], star_train['Type'])
y_pred = clf.predict(star_test[['u-g','g-r']])

In [ ]:
star_test['Predict']=y_pred
star_test

In [ ]:
star_test['Predict'].sum()/len(star_test)

In [ ]:
fig, ax = plt.subplots()
star_test[star_test['Predict']==0.0].plot.scatter('u-g','g-r',c='red',ax=ax)
star_test[star_test['Predict']==1.0].plot.scatter('u-g','g-r',c='blue',ax=ax)

# Compute the boundary
w = clf.coef_[0]
a = -w[1] / w[0]
yy = np.linspace(-0.1, 0.4)
xx = a * yy - clf.intercept_[0] / w[0]

ax.plot(xx, yy, '-k')

In [ ]:
FP = star_test[star_test['Predict']==1.0]; FP = FP[FP['Type']==0.0]
FN = star_test[star_test['Predict']==0.0]; FN = FN[FN['Type']==1.0]
TP = star_test[star_test['Predict']==1.0]; TP = TP[TP['Type']==1.0]
TN = star_test[star_test['Predict']==0.0]; TN = TN[TN['Type']==0.0]
fig, ax = plt.subplots()
TP.plot.scatter('u-g','g-r',c='blue',ax=ax,label="TP")
TN.plot.scatter('u-g','g-r',c='red',ax=ax,label="TN")
FP.plot.scatter('u-g','g-r',c='magenta',ax=ax,label="FP",marker='+',s=100)
FN.plot.scatter('u-g','g-r',c='green',ax=ax,label="FN",marker='+',s=100)

#### Radial Basis Function Kernel
We can construct a hyperplane (line) in other space by transforming data to that space, and then come back. This is done using kernels

$${\displaystyle K(\mathbf {x} ,\mathbf {x'} )=\exp \left(-{\frac {\|\mathbf {x} -\mathbf {x'} \|^{2}}{2\sigma ^{2}}}\right)}$$

In [ ]:
from scipy.ndimage import gaussian_filter
def plot_svm_rbf(gamma=20.0):
    clf_rbf = SVC(kernel='rbf', gamma=gamma)
    clf_rbf.fit(star_train[['u-g','g-r']], star_train['Type'])
    y_pred_rbf = clf_rbf.predict(star_test[['u-g','g-r']])
    star_test['PredictRBF']=y_pred_rbf
    xlim = (0.7, 1.35)
    ylim = (-0.15, 0.4)
    xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 101),
                     np.linspace(ylim[0], ylim[1], 101))
    Z = clf_rbf.predict(np.c_[ xx.ravel(),yy.ravel()])
    Z = Z.reshape(xx.shape)
    Z = gaussian_filter(Z, 2)
    FP = star_test[star_test['PredictRBF']==1.0]; FP = FP[FP['Type']==0.0]
    FN = star_test[star_test['PredictRBF']==0.0]; FN = FN[FN['Type']==1.0]
    TP = star_test[star_test['PredictRBF']==1.0]; TP = TP[TP['Type']==1.0]
    TN = star_test[star_test['PredictRBF']==0.0]; TN = TN[TN['Type']==0.0]
    fig, ax = plt.subplots()
    TP.plot.scatter('u-g','g-r',c='red',ax=ax,label="TP")
    TN.plot.scatter('u-g','g-r',c='blue',ax=ax,label="TN")
    FP.plot.scatter('u-g','g-r',c='green',ax=ax,label="FP",marker='+',s=100)
    FN.plot.scatter('u-g','g-r',c='magenta',ax=ax,label="FN",marker='+',s=100)
    ax.contour(xx, yy, Z, [0.5], colors='k')

In [ ]:
interact(plot_svm_rbf,gamma=(0.1,300,10))

## 3.2. Clustering (Unsupervised)

Now think trying to put labels but without knowing previous examples on the Galaxy data... but using now all the dimensions!

In [ ]:
gal_sample

In [ ]:
def gal_4proj(axes):
    ((ax1, ax2), (ax3, ax4)) = axes
    gal_sample.plot.scatter('u-g','redshift',color='gray',alpha=0.1,ax=ax1)
    gal_sample.plot.scatter('g-r','redshift',color='gray',alpha=0.1,ax=ax2)
    gal_sample.plot.scatter('r-i','redshift',color='gray',alpha=0.1,ax=ax3)
    gal_sample.plot.scatter('i-z','redshift',color='gray',alpha=0.1,ax=ax4)

In [ ]:
fig, axes =plt.subplots(2,2)
gal_4proj(axes)

### Gaussian Mixture Model 

Consider a Gaussian Mixture Model:
$$ \mathcal{N}(x; \mu, \Sigma) = \frac{\exp \left(-{\frac{1}{2}}( x - \mu )^{\mathrm {T}}\Sigma^{-1}(x - \mu )\right)}{\sqrt {(2\pi )^{k}|\Sigma| }}$$
$$ p(x) = \displaystyle\sum_{j=1}^{k} \phi_j\mathcal{N}(x; \mu_j, \Sigma_j)$$
$$\displaystyle\sum_{j=1}^{k} \phi_j = 1 $$

In [ ]:
from sklearn.mixture import GaussianMixture
colors = ['red','blue','green','magenta','cyan','orange']
def clust_4proj(mix,axes,n):
    for dim in range(4):
        ax = axes[int(dim/2),dim%2]
        labels=mix.predict(gal_sample)
        for i in range(n):
            gal_sample[labels==i].plot.scatter(dim,'redshift',color=colors[i],alpha=0.1,ax=ax)

In [ ]:
n=4
mix = GaussianMixture(n_components=n,covariance_type='full', max_iter=100)
mix.fit(gal_sample)
fig, axes =plt.subplots(2,2)
clust_4proj(mix,axes,n)

*Vocabulary:* This is a <font color='blue'>generative</font> <font color='green'>parametric</font> <font color='magenta'>linear</font> model for a <font color='red'>unsupervised</font> <font color='orange'>batch</font>  learning problem

In [ ]:

import matplotlib as mpl
def GMM_4proj(gmm,axes,n):
    for clust in range(n):
        for dim in range(4):
            dims=[dim,4]
            ax = axes[int(dim/2),dim%2]
            cov = gmm.covariances_[clust]
            cov = cov[dims][:,dims]
            v, w = np.linalg.eigh(cov)
            u = w[0] / np.linalg.norm(w[0])
            angle = np.arctan2(u[1], u[0])
            angle = 180 * angle / np.pi  # convert to degrees
            v = 2. * np.sqrt(2.) * np.sqrt(v)
            ell = mpl.patches.Ellipse(gmm.means_[clust,dims], v[0], v[1],
                     180 + angle, color=colors[clust])
            ell.set_clip_box(ax.bbox)
            ell.set_alpha(0.3)
            ax.add_artist(ell)

In [ ]:
def show_clusters(n=2):
    mix = GaussianMixture(n_components=n,covariance_type='full', max_iter=100)
    mix.fit(gal_sample)
    fig, axes =plt.subplots(2,2)
    gal_4proj(axes)
    GMM_4proj(mix,axes,n)

In [ ]:
interact(show_clusters,n=(2,6,1))

## 4.- Characterizing

## Dimensionality Reduction (PCA)
Consider the Singular Value Decomposition of your data (in matrix form)
$$\mathbf{X} = \mathbf{U}\mathbf{\Sigma}\mathbf{W}^T$$
Then, you can compute an affine transformation of your data such that
$${\displaystyle {\begin{aligned}\mathbf {X} ^{T}\mathbf {X} &=\mathbf {W} \mathbf {\Sigma } ^{T}\mathbf {U} ^{T}\mathbf {U} \mathbf {\Sigma } \mathbf {W} ^{T}\\&=\mathbf {W} \mathbf {\Sigma } ^{T}\mathbf {\Sigma } \mathbf {W} ^{T}\\&=\mathbf {W} \mathbf {\Sigma'}\mathbf {W} ^{T}\end{aligned}}}$$
Meaning that
$$\begin{align}
\mathbf{T} & = \mathbf{X} \mathbf{W} \\
           & = \mathbf{U}\mathbf{\Sigma}\mathbf{W}^T \mathbf{W} \\
           & = \mathbf{U}\mathbf{\Sigma}
\end{align}$$
PCA for dimensionality reduction is basically 
$$ \mathbf{T}_L = \mathbf{U}_L\mathbf{\Sigma}_L = \mathbf{X} \mathbf{W}_L  $$

In [ ]:
from sklearn import decomposition

In [ ]:
n=4
mix = GaussianMixture(n_components=n,covariance_type='full', max_iter=100)
mix.fit(gal_sample)
labels=mix.predict(gal_sample) 

In [ ]:
pca = decomposition.PCA(n_components=3)
pca.fit(gal_sample)
lowd = pca.transform(gal_sample)

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(1, figsize=(7, 5))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(lowd[:, 0], lowd[:, 1], lowd[:, 2], c=labels, cmap=plt.cm.gist_rainbow,
           edgecolor='k')

In [ ]:
pca_comp=pd.DataFrame(pca.components_)
pca_comp.columns=[['u-g', 'g-r', 'r-i', 'i-z','redshift']]
pca_comp

In [ ]:
mix = GaussianMixture(n_components=n,covariance_type='full', max_iter=100)
mix.fit(lowd)
labels_low=mix.predict(lowd)

In [ ]:
fig = plt.figure(1, figsize=(7, 5))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(lowd[:, 0], lowd[:, 1], lowd[:, 2], c=labels_low, cmap=plt.cm.gist_rainbow,
           edgecolor='k')
plt.show()